In [47]:
import json
import random
import copy

In [2]:
random.seed(42)

In [95]:
json_file ="..//data/BA_model//BA_model_all_graphs.json"
edge_mask_probability = 0.4

complete_datafile ="../data/BA_model/complete_dataset.json"
train_datafile ="../data/BA_model/train.json"
val_datafile ="../data/BA_model/val.json"
test_datafile ="../data/BA_model/test.json"

In [49]:
with open(json_file, 'r') as json_file:
    formatted_graphs = json.load(json_file)

In [40]:
formatted_graphs['0']['graph']

{'0': [11, 10, 3, 8, 4, 6, 2, 13, 7, 1, 12, 9, 5],
 '1': [0, 6, 9],
 '2': [10, 8, 6, 11, 0, 7],
 '3': [6, 0, 11, 7],
 '4': [10, 0, 9, 13, 6],
 '5': [10, 8, 7, 12, 0],
 '6': [7, 4, 2, 3, 13, 1, 8, 0],
 '7': [9, 10, 3, 2, 0, 11, 13, 5, 12, 6, 8],
 '8': [9, 0, 6, 7, 2, 5],
 '9': [8, 7, 13, 1, 12, 4, 11, 0],
 '10': [5, 7, 2, 4, 0],
 '11': [12, 3, 0, 9, 7, 2],
 '12': [11, 5, 0, 7, 9],
 '13': [7, 0, 9, 4, 6]}

In [60]:
def mask(graph):
    st = set()
    for node, neighbours in graph.items():
        node = int(node)
        for neighbor in neighbours:
            st.add((min(node,neighbor),max(node,neighbor)))
    # print("edges are  ",st)
    num_edges = len(st)
    selected_pairs_count = int(edge_mask_probability * num_edges)
    selected_pairs = random.sample(list(st), selected_pairs_count)
    # print("masking these edges  ", selected_pairs)
    for i, (first, second) in enumerate(selected_pairs):
        if random.random() < 0.5:
            selected_pairs[i] = (second, first)
    masked_graph = copy.deepcopy(graph)
    # print("original_graph is  ", graph)
    masked_nodes = set()
    for node, neighbours in masked_graph.items():
        for i,neighbor in enumerate(neighbours):
            pair = (int(node),neighbor)
            if pair in selected_pairs:
                masked_graph[node][i] = '_'
                masked_nodes.add(int(node))
    
    # print("original_graph is  ", graph)
    # print("masked_graph is  ", masked_graph)
    return masked_graph, masked_nodes

In [63]:
G=formatted_graphs['3']['graph']
# print(G)
mask(G)

({'0': [3, 1, '_'], '1': [0, '_'], '2': [0, 4], '3': [0, 4, 1], '4': [2, 3]},
 {0, 1})

In [122]:
def get_node_info(node, neighbours, out=False):
    assert len(neighbours)>0
    s=f"Node {node} is connected to node"
    if len(neighbours)>1:
        s+="s"
    s+=" "
    # if out:
    #     neighbours=sorted(neighbours)
    for i in range(len(neighbours)):
        if i==0:
            s+=str(neighbours[i])
        elif i==len(neighbours)-1:
            s+=f" and {neighbours[i]}"
        else:
            s+=f", {neighbours[i]}"
    s+=".\n"
    return s



In [123]:
get_node_info(5,[0,'_ ',2,3])

'Node 5 is connected to nodes 0, _ , 2 and 3.\n'

In [124]:
complete_dataset={}
train_data = {}
val_data = {}
test_data = {}

In [125]:
for idx, data in formatted_graphs.items():
    difficulty = data['difficulty']
    graph = data['graph']
    masked_graph, nodes_masked = mask(graph)
    while len(nodes_masked)==0:
        print("ANARTH!! ")
        masked_graph, nodes_masked = mask(graph)
    nodes=[]
    for k in graph:
        nodes.append(k)
    input= f"An undirected graph has {len(nodes)} nodes - "
    for i in range(len(nodes)):
        if i==0:
            input += str(nodes[i])
        elif i==len(nodes)-1:
            input += " and " + str(nodes[i])
        else:
            input += ", "+str(nodes[i])
    input += ".\n"
    output=""
    for node, neighbours in masked_graph.items():
        input += get_node_info(int(node),neighbours)
        if int(node) in nodes_masked:
            output += get_node_info(int(node), graph[node] )
    # print(input)
    # print("------------")
    # print(output)
    complete_dataset[len(complete_dataset)]={"input":input , "output":output}
    if random.random()<0.8:
        train_data[len(train_data)]={"input":input , "output":output}
    elif random.random() < 0.5:
        val_data[len(val_data)]={"input":input , "output":output}
    else:
        test_data[len(test_data)]={"input":input , "output":output}
    
    

In [126]:
with open(complete_datafile,'w') as json_file:
    json.dump(complete_dataset, json_file)

In [127]:
with open(train_datafile,'w') as json_file:
    json.dump(train_data, json_file)

In [128]:
with open(val_datafile,'w') as json_file:
    json.dump(val_data, json_file)
with open(test_datafile,'w') as json_file:
    json.dump(test_data, json_file)

In [135]:
print(complete_dataset[10]['output'])

Node 0 is connected to nodes 7, 2, 4, 1, 3 and 5.
Node 1 is connected to nodes 4, 6, 0 and 5.
Node 6 is connected to nodes 1 and 3.

